In [4]:
import numpy as np
import pandas as pd


from sklearn.manifold import TSNE
from sklearn.manifold import MDS

from IPython.display import display
import logging as log
import plotly
from plotly.offline import init_notebook_mode, iplot
plotly.offline.init_notebook_mode() # run at the start of every ipython notebook
plotly.plotly.sign_in('spersad', 'MwbO3xbqh2Mv6sfhCma7')
import plotly.graph_objs as go

log.getLogger().setLevel(log.INFO)

import tensorflow as tf

AttributeError: module 'pandas' has no attribute 'plotting'

### Simulate binary vectors


In [ ]:
def load_bitvectors(paths):
    '''
    Given a pandas dataframe with headings Read_name Binary_vector N_mutations Reference_name Start_position
    load only the bitvectors as a numpy array of integers
    @param: paths - a list of paths to bitvector file for each cluster
    @return: bitreads - an array of bitvectors, not corrected for correlation between bases
    '''
    
    for path in paths:
        df = pd.DataFrame(path, sep=' ')
        print(df.head())
    return bitreads

def denoise_bitvectors(bitvectors, threshold=0.001):
    '''
    Given an array of bitvectors, zero out all columns where the population average is below the given threshold.
    @param: bitvectors - numpy array of bitvectors
    @param: decimal threshold below which entries are considered noise. Default to 0.001=0.1%
    
    @return: denoised_bitvectors - numpy array of bitvectors where low signal positions are zero-ed out.'''
    population_average = bitvectors.sum(axis=0)/bitvectors.shape[1]
    denoise = np.where(population_average<threshold)
    bitvectors[:,denoise] = 0
    
    return bitvectors


In [ ]:
load_bitvectors(['riboA_sim.txt','riboC_sim.txt'])


In [14]:
def second_order_distance(dist_matrix, k):
    ''' Given a distance matrix, compute the second order distance matrix,
        where the distance between i, j is the d(i,j) + alpha Sum(d(i,k_j)+d(k_i,j))
        and k_j is the k-NN neighborhood of j and k_i is the k-NN neighborhood of i'''

In [ ]:
riboX, riboY, riboT = load_ribosnitch()
log.info('Loaded ribosnitch')

riboY

In [9]:
df = pd.DataFrame(simX4)
print(df.shape)
df.drop_duplicates(inplace=True)
print(df.shape)

(8175, 400)
(7999, 400)


In [10]:
simX = df.as_matrix()
simX.shape

(7999, 400)

In [25]:
def mask_dependent_bases(X):
    print(X.dtype)
    X_pad = np.lib.pad(X.astype(np.float64), (3,3), 'constant', constant_values=(0.0, 0.0))[3:-3]
    X_pad = X_pad.reshape(1,-1)[0]
    ones = np.where(X_pad==1)[0]
    X_pad[ones-1] = np.nan
    X_pad[ones-2] = np.nan
    X_pad[ones-3] = np.nan
    
    X_pad[ones+1] = np.nan
    X_pad[ones+2] = np.nan
    X_pad[ones+3] = np.nan
    X_masked = X_pad.reshape(-1,X.shape[1]+6)
    
    return X_masked[:,3:-3]

def dist_matrix(X):
    dist = np.zeros((X.shape[0],X.shape[0]))
    print(dist.shape)
    for i in range(len(X)):
        for j in range(i, len(X)):
            row1 = X[i]
            row2 = X[j]
            d = np.nansum(np.abs(row1-row2))
            dist[i,j] = d
            dist[j,i] = d
    return dist


log.info('Masking X1 bases')
masked_X4 = mask_dependent_bases(simX[:5])
pdist_x4 = dist_matrix(masked_X4)

INFO:root:Masking X1 bases


int32
(5, 5)


In [40]:
masked_X4

array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,  nan,   1.,  nan],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]])

In [26]:
pdist_x4

array([[  0.,  11.,   5.,   6.,  10.],
       [ 11.,   0.,   8.,   9.,  11.],
       [  5.,   8.,   0.,   7.,   9.],
       [  6.,   9.,   7.,   0.,  10.],
       [ 10.,  11.,   9.,  10.,   0.]])

In [ ]:
def second_order_distance(X, k=5):
    ''' Given a set of reads, compute the second order distance matrix,
        where the distance between reads i, j is the d(i,j) + alpha Sum(d(i,k_j)+d(k_i,j))
        and k_j is the k-NN neighborhood of j and k_i is the k-NN neighborhood of i'''
    ALPHA=0.5
    def mydist(x,y):
        print(x)
        print(y)
        print(x-y)
        assert False
        return np.nansum(np.abs(x-y))
    
    from sklearn.neighbors import NearestNeighbors as NN
    nbrs  = NN().fit(X) #(algorithm='ball_tree', metric=mydist
    dists, indices = nbrs.kneighbors(n_neighbors=k)
    print(dists.shape)
    indices = indices[:,1:k] # Each read is not in its own nearest neighborhood
    
    second_dist = np.zeros((X.shape[0],X.shape[0]))
    print(second_dist.shape)
    for i in range(len(X)):
        for j in range(i, len(X)):
            row1 = X[i]
            row2 = X[j]
            d = np.nansum(np.abs(X[i]-X[j])) + ALPHA*(np.nansum(np.abs(X[indices[i]] - X[j])) + np.nansum(np.abs(X[indices[j]] - X[i])))
            second_dist[i,j] = d
            second_dist[j,i] = d
    return second_dist

dists = second_order_distance(simX[:], k=5)

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 1 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
(7999, 5)
(7999, 7999)


In [ ]:
def clean_experimental(path):
    for label in ['A','C']:
        bitvectors=pd.read_csv(path,sep=' ')
        bitvectors = bitvectors[bitvectors['N_mutations']==label]
        #Grab the bit vectors only 
        bitreads = bitvectors.as_matrix(columns=['Binary_vector'])
        log.debug("bitreads[0][0]=%s" % ( bitreads[0][0]))
        len_bits = str(len(list(bitreads[0][0])))
        log.debug("Len bits is %s" % len_bits)
        size = bitreads.size
        bitreads = np.array(bitreads,dtype='|S'+len_bits)
        bitreads = bitreads.view('S1')
        bitreads = bitreads.reshape((size, -1))
        bitreads = bitreads.astype('|S4')
        bitreads[bitreads == b'?'] = np.nan    
        bitreads = bitreads.astype(np.float)
        reads_df = pd.DataFrame(bitreads)#.astype(int)
        print(reads_df.head())
        reads_df.to_csv('ribo{0}_real.txt'.format(label),float_format='%.0f', na_rep='?', index=False, sep='\t')
x = clean_experimental('ribo_real.txt')

<hr>
#### t-SNE Visualization

Now, we wish to visualize our cluster data when embedded in a lower dimensional space.



In [45]:
def performTSNE(X, Y, num_examples=2000, perp=30, metric='euclidean'):

    print(X.shape)
    model = TSNE(n_components=2, perplexity = perp, random_state=0, metric=metric) # fit into 2D space
    log.info('Defined model')
    if metric !='precomputed':
        X = X[:num_examples]
        Y = Y[:num_examples]
        
    embeddedX = model.fit_transform(X)
    log.info('Created embedding')

    # Scatter plot to visualize embedded data
    # Create a trace
    trace = go.Scatter(
        x = embeddedX[:,0],
        y = embeddedX[:,1],
        mode = 'markers',
        marker=dict(
            size='2',
            color = Y, # color points by label they belong to
            colorscale= [[0, '#dd2c4f'], [1, '#3d6fcc']],
        )
    )

    data = [trace]
    
    layout = go.Layout(
        title='Embedding of Clusters in 2D Space',
    )

    fig = go.Figure(data=data, layout=layout)
    iplot(fig, filename='t-SNE-embedding')
    
    log.info('Plotted data')


In [22]:
def performMDS(X, Y, num_examples=1000, perp=30, metric='euclidean'):

    print(X.shape)
    model = MDS(n_components=2, max_iter=3000, dissimilarity=metric)
    log.info('Defined model')
    
    if metric !='precomputed':
        X = X[:num_examples]
        Y = Y[:num_examples]
        
    embeddedX = model.fit_transform(X)
    log.info('Created embedding')

    # Scatter plot to visualize embedded data
    # Create a trace
    trace = go.Scatter(
        x = embeddedX[:,0],
        y = embeddedX[:,1],
        mode = 'markers',
        marker=dict(
            size='2',
            color = Y, # color points by label they belong to
            colorscale= [[0, '#dd2c4f'], [1, '#3d6fcc']],
        )
    )

    data = [trace]
    
    layout = go.Layout(
        title='Embedding of Clusters in 2D Space',
    )

    fig = go.Figure(data=data, layout=layout)
    iplot(fig, filename='MDS-embedding')
    
    log.info('Plotted data')
    

In [23]:
performMDS(simX4, simY4)

INFO:root:Defined model


(8147, 400)


INFO:root:Created embedding


INFO:root:Plotted data


In [51]:
performTSNE(dists, simY4,metric='precomputed')

INFO:root:Defined model


(8185, 400)


INFO:root:Created embedding


INFO:root:Plotted data


In [ ]:
performTSNE(simX4, simY4, num_examples=10000)

In [48]:
masked_X4 = mask_dependent_bases(simX4)

int32
1
2
6


In [49]:
pdist_X4 = dist_matrix(masked_X4)

(8185, 8185)


In [50]:
performTSNE(pdist_X4, simY4, metric='precomputed')

INFO:root:Defined model


(8185, 8185)


INFO:root:Created embedding


INFO:root:Plotted data


<hr>

#### PCA

Now we wish to see whether PCA can recover the correct number of clusters in our data, as well as whether the dimensionality reduction yields cluster separation



In [ ]:
def performPCA(X, Y, num_examples=2000):
    from sklearn.decomposition import PCA

    pca = PCA()
    log.info('Defined PCA Model')
    reducedX = pca.fit_transform(X[:num_examples])
    log.info('Fit X to Model')

    # Create a trace
    trace = go.Scatter(
        x = reducedX[:,0],
        y = reducedX[:,1],
        mode = 'markers',
        marker=dict(
            size='2',
            color = Y[:num_examples], # color points by label they belong to
            colorscale= [[0, '#dd2c4f'], [1, '#3d6fcc']],
        )
    )

    data = [trace]

    # Plot and embed in ipython notebook!
    iplot(data, filename='basic-scatter')



In [ ]:
performPCA(simX1, simY1)

In [ ]:
performPCA(simX2, simY2)

In [ ]:
performPCA(simX3, simY3)

In [ ]:
performPCA(simX4, simY4)

In [ ]:
performPCA(riboX,riboY)

In [19]:
def run_autoencoder(simX, simY, n_hidden_1=256, n_hidden_2=128, learning_rate=0.01, training_epochs=2000, BATCH_SIZE=200, draw_examples=False, draw_dim1=20,draw_dim2=20):
    import matplotlib.pyplot as plt
    %matplotlib inline

    # Parameters
    display_step = 100
    n_input = simX.shape[1]   # length of input read
    
    X = tf.placeholder("float64", [None, n_input])

    encoder_W1 = tf.Variable(tf.random_normal([n_input, n_hidden_1], dtype="float64"), name="encoder_W1")
    encoder_W2 =  tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], dtype="float64"), name="encoder_W2")
    weights = {
        'decoder_h1': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_1], dtype="float64")),
        'decoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_input], dtype="float64")),
    }

    encoder_b1 = tf.Variable(tf.random_normal([n_hidden_1], dtype="float64"), name="encoder_b1"),
    encoder_b2 = tf.Variable(tf.random_normal([n_hidden_2], dtype="float64"), name="encoder_b2"),
    biases = {
        'decoder_b1': tf.Variable(tf.random_normal([n_hidden_1], dtype="float64")),
        'decoder_b2': tf.Variable(tf.random_normal([n_input], dtype="float64")),
    }


    # Building the encoder
    def encoder(x):
        # Encoder Hidden layer with sigmoid activation #1
        layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, encoder_W1),
                                       encoder_b1))
        # Decoder Hidden layer with sigmoid activation #2
        layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, encoder_W2),
                                       encoder_b2))
        return layer_2


    # Building the decoder
    def decoder(x):
        # Encoder Hidden layer with ReLU activation #1
        layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                       biases['decoder_b1']))
        # Decoder Hidden layer with ReLU activation #2
        layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                       biases['decoder_b2']))
        return layer_2

    # Construct model
    encoder_op = encoder(X)
    decoder_op = decoder(encoder_op)

    # Prediction
    y_pred = decoder_op
    # Targets (Labels) are the input data.
    y_true = X

    # Define loss and optimizer, minimize the squared error
    cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
    optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)

    # Initializing the variables
    init = tf.global_variables_initializer()
    
    NUM_BATCHES = simX.shape[0]//BATCH_SIZE

    costs = []
    # Launch the graph
    with tf.Session() as sess:
        sess.run(init)
        # Training cycle
        log.info('Training Autoencoder')
        for epoch in range(training_epochs):
            # Loop over all batches
            for i in range(NUM_BATCHES):
                batch_xs= simX[BATCH_SIZE*i : BATCH_SIZE*(i+1)]
                # Run optimization op (backprop) and cost op (to get loss value)
                _, c = sess.run([optimizer, cost], feed_dict={X: batch_xs})
                
            # Display logs per epoch step
            if epoch % display_step == 0:
                print("Epoch:", '%04d' % (epoch),
                      "cost=", "{:.9f}".format(c))
                costs.append(c)

        log.info("Optimization Finished!")
         # Applying encode and decode over test set
        encoded, encode_decode = sess.run(
            [encoder_op, y_pred], feed_dict={X: simX})
        
        if draw_examples:
            examples_to_show = 10
            # Compare original images with their reconstructions
            f, a = plt.subplots(2, examples_to_show, figsize=(examples_to_show, 2))
            for i in range(examples_to_show):
                a[0][i].imshow(np.reshape(simX[i], (draw_dim1,draw_dim2)))
                a[1][i].imshow(np.reshape(encode_decode[i], (draw_dim1,draw_dim2)))
            plt.draw()

        encode_W1_vals = sess.run([v for v in tf.global_variables() if v.name == "encoder_W1:0"][0])
        encode_W2_vals = sess.run([v for v in tf.global_variables() if v.name == "encoder_W2:0"][0])
        
        encode_b1_vals = sess.run([v for v in tf.global_variables() if v.name == "encoder_b1:0"][0])
        encode_b2_vals = sess.run([v for v in tf.global_variables() if v.name == "encoder_b2:0"][0])
     
    log.info('Encoded data has dimension {0} and {1} samples.'.format(encoded.shape[1],encoded.shape[0]))
    if draw_examples:
        ## Perform t-SNE to determine whether the embedded points are still close in the new low dimensional space
        performTSNE(encoded, simY)
    return costs, encoded, n_hidden_2, encode_W1_vals, encode_W2_vals, encode_b1_vals, encode_b2_vals


In [ ]:
ae_c1, ae_e1, _, _, _, _, _ = run_autoencoder(simX1[:2000], simY1[:2000], draw_examples=True)

In [ ]:
ae_c2, ae_e2, _, _, _, _, _ = run_autoencoder(simX2[:2000], simY2[:2000], draw_examples=True)

In [ ]:
ae_c3, ae_e3, _, _, _, _, _ = run_autoencoder(simX3[:2000], simY3[:2000], draw_examples=True)

In [ ]:
ae_c4, ae_e4, _, _, _, _, _ = run_autoencoder(simX4[:2000], simY4[:2000], draw_examples=True)

In [ ]:
# Plot the cost function for learning each sample
epochs = [i for i in range(len(ae_c1))]
trace1 = go.Scatter(
    x = epochs,
    y = ae_c1,
    name = 'Sample 1',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 4,)
)
trace2 = go.Scatter(
    x = epochs,
    y = ae_c2,
    name = 'Sample 2',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4,
    )
)
trace3 = go.Scatter(
    x = epochs,
    y = ae_c3,
    name = 'Sample 3',
    line = dict(
        color = ('rgb(232, 172, 53)'),
        width = 4,
        )
)
trace4 = go.Scatter(
    x = epochs,
    y = ae_c4,
    name = 'Sample 4',
    line = dict(
        color = ('rgb(28, 158, 73)'),
        width = 4,
    )
)
    
data=[trace1,trace2,trace3,trace4]

iplot(data)

### Deep Embedded Clustering

We discard the decoding function from our autoencoder, and use the learned encoding weights to map our data from the higher dimensional space into our feature space


In [20]:
def run_DEC(simX, simY, encoding=True, learning_rate = 1e-2, N_CLUSTERS=2, ALPHA=1, DEC_training_steps=2000, draw_examples=False, draw_dim1=20, draw_dim2=20):

    tf.reset_default_graph()
  
    n_samples = simX.shape[0]

    # Create a placeholder for inputting training X data 
    input_X = tf.placeholder("float64", [None, simX.shape[1]], name="Input_X")

    if encoding=='ae':
        # Run auto-encoder to generate embedded data
        costs, encoded, DIM_Z, encode_W1_vals, encode_W2_vals, encode_b1_vals, encode_b2_vals = run_autoencoder(simX,simY,draw_examples=draw_examples,draw_dim1=draw_dim1, draw_dim2=draw_dim2)

        # Building the encoder
        encoder_W1 = tf.Variable(encode_W1_vals, name="encoder_W1")
        encoder_W2 =  tf.Variable(encode_W2_vals, name="encoder_W2")
        encoder_b1 = tf.Variable(encode_b1_vals, name="encoder_b1"),
        encoder_b2 = tf.Variable(encode_b2_vals, name="encoder_b2"),   
        def encoder(x):
            # Encoder Hidden layer with sigmoid activation #1
            layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, encoder_W1),
                                        encoder_b1))
            # Decoder Hidden layer with sigmoid activation #2
            layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, encoder_W2),
                                           encoder_b2))
            return layer_2
        X = encoder(input_X)

    elif encoding=='tsne':
        print('Performing TSNE')
        DIM_Z = 2
        from sklearn.manifold import TSNE
        model = TSNE(n_components=N_CLUSTERS, perplexity = 30, random_state=0) # fit into 2D space
        encoded = model.fit_transform(simX)
        X = encoded
    else:
        X = input_X
        encoded = simX
        DIM_Z = simX.shape[1]
        
    # Initialize mu by k means, return a numpy array of shape (N_CLUSTERS, DIM_Z)
    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=0).fit(encoded)
    mu_init = kmeans.cluster_centers_
    
#     #Debugging mu_init
#     mu_init = np.array([[0,1],[4,5]])

    print('Kmeans accuracy: {0}'.format(100*sum(kmeans.predict(encoded)==simY)/len(simY)))
    print('Initialize {0} clusters with dimensions {1}'.format(N_CLUSTERS, encoded.shape[1]))

    # Create another Variable initialized with the original cluster centers.
    Mu = tf.Variable(mu_init, name="Input_Mu", dtype="float64")
    
    
    X_1 = tf.reshape(X, [n_samples, DIM_Z]) 
    c = tf.tile(X_1, tf.constant([1, N_CLUSTERS]))
    X_reshape = tf.reshape(c,[n_samples,N_CLUSTERS,DIM_Z], name="X_reshape")

    #Reshape Mu into a dimension befitting the number of data points
    Mu_tile = tf.tile(Mu, tf.constant([n_samples, 1]), name="Mu_tile")
    Mu_reshape = tf.reshape(Mu_tile, [n_samples,N_CLUSTERS,DIM_Z], name="Mu_reshape")
    a = tf.squared_difference(Mu_reshape, X_reshape)
    b = tf.reduce_sum(tf.squared_difference(Mu_reshape, X_reshape), 2)
    Q = tf.pow(tf.reduce_sum(tf.squared_difference(Mu_reshape, X_reshape), 2)/ALPHA + 1, -0.5*(ALPHA+1))

    Q_norm = Q/tf.reduce_sum(Q, 1, keep_dims=True) ## FINAL Q_ij matrix, where position (i,j) is q_ij

    # Compute auxiliary probability distribution P

    # p_ij = q_ij^2/f_j , where f_j is the sum of q_ij over i (column-sum)
    P = tf.pow(Q_norm,2)/ tf.reduce_sum(Q_norm, 0)
    # Now normalize by cluster (row)
    P_norm = P/tf.reduce_sum(P, 1, keep_dims=True) ## This gives the responsibilities of each cluster

    KL_matrix = tf.multiply(P_norm, tf.log(tf.div(P_norm, Q_norm)))
    cost = -(tf.reduce_sum(KL_matrix))

    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

    # Initializing the variables
    init = tf.global_variables_initializer()

    costs = []
    similarities = []
    probabilities = []
    clusters = []
    # Launch the graph
    with tf.Session() as sess:
        sess.run(init)
        for i in range(DEC_training_steps):
            _, c, x, p, m_t, m_r, m, q_norm = sess.run([optimizer, cost, X, P_norm,Mu_tile, Mu_reshape, Mu, Q_norm], feed_dict={input_X:simX})
            costs.append(c)
            if (i%100==1):
                print('Cost at epoch {0} : {1}'.format(i, c))
                preds = np.argmax(p, axis=1)
                acc = 100*sum(simY==preds)/len(preds)
                print('Accuracy :{0}'.format(acc))
                probabilities.append(p)
                clusters.append(m)
    print('Optimization complete!')
    preds = np.argmax(probabilities[-1], axis=1)
    acc = 100*sum(simY==preds)/len(preds)
    print('Final accuracy: {0}'.format(acc))
    return costs, probabilities, clusters, x

In [ ]:
cost1, prob1, cluster1, x1 = run_DEC(masked_X1[:2000], simY1[:2000], encoding='ae', DEC_training_steps=2000)
# c, p, mu, x = run_DEC(np.array([[1,2],[1,3],[5,6],[6,6]]), np.array([0,0,1,1]), encoding='none', DEC_training_steps=2000)


In [ ]:
cost2, prob2, cluster2, x2 = run_DEC(simX2[:2000], simY2[:2000], encoding='ae', DEC_training_steps=500)

In [ ]:
cost3, prob3, cluster3, x3 = run_DEC(simX3[:2000], simY3[:2000], encoding='ae')

In [ ]:
cost4, prob4, cluster4, x4 = run_DEC(simX4[:2000], simY4[:2000], encoding='ae')

In [ ]:
cost_ribo, prob_ribo, cluster_ribo, x_ribo = run_DEC(riboX[:2000], riboY[:2000], encoding='ae', draw_examples=True, draw_dim1=8, draw_dim2=5)

#### Analyse Cluster Results

In [ ]:
def extract_cluster_avg(simX, prob, p0, p1):
    ''' 
    Plot the recovered cluster average versus the true cluster probabilities 
    '''
    
    from plotly import tools
    
    preds = np.argmax(prob[-1], axis=1)
    zeros = (np.where(preds==0)[0])
    ones = (np.where(preds==1)[0])
    
    X0 = simX[zeros]
    X1 = simX[ones]
    
    X0_avg = np.mean(X0, axis=0)
    X1_avg = np.mean(X1, axis=0)
    
    # Create a trace
    trace1 = go.Scatter(
        x = X0_avg,
        y = p0,
        mode = 'markers',
        name = "Cluster 1"
    )

    trace2 = go.Scatter(
        x = X1_avg,
        y = p1,
        mode = 'markers',
        name = "Cluster 2"
    )
    
    fig = tools.make_subplots(rows=1, cols=2)

    fig.append_trace(trace1, 1, 1)
    fig.append_trace(trace2, 1, 2)

    # Plot and embed in ipython notebook!
    iplot(fig, filename='basic-scatter')


    return X0, X1



In [ ]:
ribo_p0 = np.mean(riboX[riboY==0], axis=0)
ribo_p1 = np.mean(riboX[riboY==1], axis=0)
extract_cluster_avg(riboX, prob_ribo, ribo_p1, ribo_p0)

In [ ]:
def plot_violin(cluster, simY, one=1, zero=0):
    from plotly import tools
    from plotly.tools import FigureFactory as ff

    ones = cluster[np.where(simY == 1)]
    zeros = cluster[np.where(simY==0)]
    
    c0_dist = zeros[:,zero]
    c1_dist = ones[:,one]

    fig0 = ff.create_violin(c0_dist, colors='#1F77B4')
    fig1 = ff.create_violin(c1_dist, colors='#D62728')
    
    #create a subplot with a shared y-axis
    fig = tools.make_subplots(rows=1, cols=2)

    for f in fig0.data:
        fig.append_trace(f, 1, 1)

    for f in fig1.data:
        fig.append_trace(f, 1, 2)

    iplot(fig)

    return None

In [ ]:
plot_violin(prob_ribo[-1], riboY[:2000], zero=1, one=0)

In [ ]:
def extract_wrong_points(X, Y, prob, zero=0, one=1):
    '''
    Given the true labels, extract some wrong samples
    '''
    preds = np.argmax(prob[-1], axis=1)
    zeros = np.where(preds==zero)[0]
    ones = np.where(preds==one)[0]
    
    wrong = np.where(Y!=preds)
    
    print(prob[-1][wrong])
    
    
extract_wrong_points(riboX[:2000], riboY[:2000], prob2)
    